# Swap Samples

This notebook demonstrates how to access and use the financial-contracts end point of Instrument Pricing Analytics (IPA) in the Refinitiv Data Platform.

##### Learn more

Please visit the [Refinitiv Developer Community Portal](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-ir-swap-contracts) to access the API documentation and much more.

You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

##### Getting Help and Support

If you have any questions regarding the API usage, please post them on the [Refinitiv Data Platform Q&A Forum](https://community.developers.refinitiv.com/spaces/231/index.html). The Refinitiv Developer Community will be very pleased to help you. 

#### Import the library and connect to the Refinitv Data Platform

This sample works in both desktop and non-desktop environment. The appropriate session can be created based on the constant variable IPA_ENVIRONMENT, that is set by default to DESKTOP for execution in the Refinitiv codebook application. To run this sample in a non-desktop environment, you may set the variable to NON-DESKTOP.

In [3]:
import refinitiv.dataplatform as rdp

# Set IPA_ENVIRONMENT to NON-DESKTOP to use this example in a non desktop environment
#IPA_ENVIRONMENT = "NON-DESKTOP"
IPA_ENVIRONMENT = "DESKTOP"

if (IPA_ENVIRONMENT == "DESKTOP"):
    session = rdp.DesktopSession(app_key='DEFAULT_CODE_BOOK_APP_KEY')
else:
    %run ".\..\..\credentials.ipynb"
    session = rdp.PlatformSession(APP_KEY, rdp.GrantPassword(RDP_LOGIN, RDP_PASSWORD))

session.open()
endpoint = rdp.Endpoint(session, "https://api.refinitiv.com/data/quantitative-analytics/v1/financial-contracts")
print("session opened and end-point set")

session opened and end-point set


### Linear Amortization

This sample generate the cash flows of an interest rate swap whose principal amortizes linearily. The 'fields' requested are:
- "CashFlowDatesArray": the cash flow payment date
- "CashFlowInterestAmountsInDealCcyArray" the interet amount paid or received
- "CashFlowResidualAmountsInLegCcyArray", the remaining pricipal at each payment date

the amortization parameters used are the following:
- "AmortizationFrequency": the frequency of amortization (EveryCoupon, Every2ndCoupon, Every3rdCoupon, Every4thCoupon, Every12thCoupon)
- "AmortizationType": Linear, Annuity, Schedule or none.
- "remainingNotional": some residual principal at the last amortization date on top of the scheduled payment

In [46]:
request_body = {
    
    "fields" : ["InstrumentTag","Direction","CashFlowDatesArray","CashFlowInterestAmountsInDealCcyArray","CashFlowResidualAmountsInLegCcyArray",
                "DiscountCurveName","ForwardCurveName","ErrorCode","ErrorMessage"],
    
    "universe" : [
        {
            "instrumentType":"Swap",
            "instrumentDefinition": {
                "instrumentTag":"IRS-EURIBOR-5Y",
                "startDate":"2020-01-28",
                "tenor":"5Y",
                "legs":[
                {
                    "notionalAmount":10000000.0,
                    "direction":"Paid",
                    "interestType":"Fixed",
                    "fixedRatePercent":0.5,
                    "notionalCcy":"EUR",
                    "interestPaymentFrequency":"Annual",
                    "interestCalculationMethod":"Dcb_30_360",
                    "AmortizationSchedule": [{
                        "remainingNotional":0.0,
                        "AmortizationFrequency": "EveryCoupon",
                        "AmortizationType": "Linear"
                    }]
                },
                {
                    "notionalAmount":10000000.0,
                    "direction":"Received",
                    "interestType":"Float",
                    "interestPaymentFrequency":"SemiAnnual",
                    "interestCalculationMethod":"Dcb_Actual_360",
                    "notionalCcy":"EUR",
                    "indexName":"EURIBOR",
                    "indexTenor":"6M",
                    "AmortizationSchedule": [{
                        "remainingNotional":1000000.,
                        "AmortizationFrequency": "Every2ndCoupon",
                        "AmortizationType": "Linear"
                    }]
                    
                }]
            }
        }],
    
    "pricingParameters": {
        "valuationDate": "2020-10-20T00:00:00Z"
    },
        
    "outputs" : ["Data","Headers"],

}

response = endpoint.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body)

print(response.data.raw)

{'headers': [{'type': 'String', 'name': 'InstrumentTag'}, {'type': 'String', 'name': 'Direction'}, {'type': 'DateArray', 'name': 'CashFlowDatesArray'}, {'type': 'FloatArray', 'name': 'CashFlowInterestAmountsInDealCcyArray'}, {'type': 'FloatArray', 'name': 'CashFlowResidualAmountsInLegCcyArray'}, {'type': 'String', 'name': 'DiscountCurveName'}, {'type': 'String', 'name': 'ForwardCurveName'}, {'type': 'String', 'name': 'ErrorCode'}, {'type': 'String', 'name': 'ErrorMessage'}], 'data': [['IRS-EURIBOR-5Y', 'Paid', ['2021-01-28T00:00:00Z', '2022-01-28T00:00:00Z', '2023-01-30T00:00:00Z', '2024-01-29T00:00:00Z', '2025-01-28T00:00:00Z'], [-50000.0, -40000.0, -30166.666666666668, -19944.444444444445, -9972.222222222223], [8000000.0, 6000000.0, 4000000.0, 2000000.0, 0.0], 'EUR - Swap vs 6M Euribor', None, '', ''], ['IRS-EURIBOR-5Y', 'Received', ['2020-07-28T00:00:00Z', '2021-01-28T00:00:00Z', '2021-07-28T00:00:00Z', '2022-01-28T00:00:00Z', '2022-07-28T00:00:00Z', '2023-01-30T00:00:00Z', '2023-07

In [47]:
import pandas as pd

headers_name = [h['name'] for h in response.data.raw['headers']]
df = pd.DataFrame(data=response.data.raw['data'], columns=headers_name)
df

,InstrumentTag,Direction,CashFlowDatesArray,CashFlowInterestAmountsInDealCcyArray,CashFlowResidualAmountsInLegCcyArray,DiscountCurveName,ForwardCurveName,ErrorCode,ErrorMessage
0,IRS-EURIBOR-5Y,Paid,"[2021-01-28T00:00:00Z, 2022-01-28T00:00:00Z, 2...","[-50000.0, -40000.0, -30166.666666666668, -199...","[8000000.0, 6000000.0, 4000000.0, 2000000.0, 0.0]",EUR - Swap vs 6M Euribor,None,,
1,IRS-EURIBOR-5Y,Received,"[2020-07-28T00:00:00Z, 2021-01-28T00:00:00Z, 2...","[35233.758447940054, 33065.38705115308, 21543....","[10000000.0, 8200000.0, 8200000.0, 6400000.0, ...",EUR - Swap vs 6M Euribor,EUR - Swap vs 6M Euribor,,


In [48]:
paid_cash_flow_df = pd.DataFrame()

paid_cash_flow_df['Paid Dates'] = df[df["Direction"] == "Paid"]['CashFlowDatesArray'][0]
paid_cash_flow_df['Paid InterestPercent'] = df[df["Direction"] == "Paid"]['CashFlowInterestAmountsInDealCcyArray'][0]
paid_cash_flow_df['Paid Principal'] = df[df["Direction"] == "Paid"]['CashFlowResidualAmountsInLegCcyArray'][0]

paid_cash_flow_df

,Paid Dates,Paid InterestPercent,Paid Principal
0,2021-01-28T00:00:00Z,-50000.000000,8000000.0
1,2022-01-28T00:00:00Z,-40000.000000,6000000.0
2,2023-01-30T00:00:00Z,-30166.666667,4000000.0
3,2024-01-29T00:00:00Z,-19944.444444,2000000.0
4,2025-01-28T00:00:00Z,-9972.222222,0.0


In [49]:
received_cash_flow_df = pd.DataFrame()

received_cash_flow_df['Rec Dates'] = df[df["Direction"] == "Received"]['CashFlowDatesArray'][1]
received_cash_flow_df['Rec InterestPercent'] = df[df["Direction"] == "Received"]['CashFlowInterestAmountsInDealCcyArray'][1]
received_cash_flow_df['Rec Principal'] = df[df["Direction"] == "Received"]['CashFlowResidualAmountsInLegCcyArray'][1]

received_cash_flow_df

,Rec Dates,Rec InterestPercent,Rec Principal
0,2020-07-28T00:00:00Z,35233.758448,10000000.0
1,2021-01-28T00:00:00Z,33065.387051,8200000.0
2,2021-07-28T00:00:00Z,21543.631589,8200000.0
3,2022-01-28T00:00:00Z,20754.927631,6400000.0
4,2022-07-28T00:00:00Z,15738.303058,6400000.0
5,2023-01-30T00:00:00Z,16512.211809,4600000.0
6,2023-07-28T00:00:00Z,11916.046080,4600000.0
7,2024-01-29T00:00:00Z,13017.886109,2800000.0
8,2024-07-29T00:00:00Z,8346.264126,2800000.0
9,2025-01-28T00:00:00Z,9076.195896,0.0
